In [7]:
import time,serial
import numpy as np

def servoin(a=90,b=90,c=90,d=90,e=90,f=90,delay=3):
    '''waits end of previous act. and send angles to arduino returns acc-array     
    '''
    while ser.read()!=b'>':pass
    acc = [float(e) for e in ser.readline().split(b'\t')]
    print(acc)
    with open('historia.log', 'a') as myfile:
            myfile.write('Accelerometer:'+str(acc)+'\n')
    if fall_check(*acc)==False:
        data='in'
        for i in a,b,c,d,e,f,delay:
            try: 
                data+=str(hex(i))[2]+str(hex(i))[3]
            except IndexError:
                data+='0'+str(hex(i))[2]
        ser.write(data.encode())
        print('servo in: ',a,b,c,d,e,f,delay)
        with open('historia.log', 'a') as myfile:
            myfile.write('Servo in:'+str([a,b,c,d,e,f,delay])+'\n')
    else:
         1/0
    return acc
    
def fall_check(accelx,accely,accelz,gx,gy,gz):
    '''gets measurement of accelerometer
        checks if Robo has fallen
        returns FALSE if Robo - NOT fallen
        returns TRUE if Robo - has fallen'''
    if (abs(accely)>5 or abs(accelz)>5) and abs(accelx)<7:return True
    else: return False

def output(s):
    print(s)
    with open('historia.log', 'a') as myfile:
            myfile.write(s)

def serial_begin(port):
    ser = serial.Serial(port, 19200, bytesize=8, parity='N', stopbits=1, timeout=2)
    time.sleep(2)
    output(time.ctime()+'\nconnected to: '+ ser.portstr)
    return ser,np.array([[90,90,90,90,90,90]])

def executor(ser,prt,mtrx):
    k = 0
    acc = []
    try:#fall detection script            
        for i in mtrx: 
            acc.append(servoin(*i))
            k += 1
    except ZeroDivisionError:
        output('I have fallen.\n')
        stand_up([90,90,90,90,90,90])
    return ser,k,acc

def stand_up(old):
    """TRUE when ready"""
    try:
        data='in'
        for i in old:
            try: 
                data+=str(hex(i))[2]+str(hex(i))[3]
            except IndexError:
                data+='0'+str(hex(i))[2]
        data+='03'
        ser.write(data.encode())
        servoin(*old)
    except ZeroDivisionError:
        print('Master, lift me up, please...')
        stand_up(old)

if __name__ == "__main__":
    port = 'COM9'
    ser,matrix = serial_begin(port)
    matrix = np.loadtxt('matrix.txt', 'int')
    ser,stf,sensor_data = executor(ser,port,matrix)
    if stf < len(matrix): output('Steps to fall :'+str(stf))
    output('Steps to fall:'+str(stf)+'\n')
    np.savetxt('sensor.txt',sensor_data,fmt='%.2f',delimiter='\t')
    output('[Upd]sensor.txt\nSensor data recieved.\n')    
    ser.close()            

Mon Nov 25 21:23:15 2019
connected to: COM9
[-8.28, -1.69, -0.67, 0.0, 0.0, 0.0]
servo in:  89 82 61 89 114 86 3
[-10.67, 0.43, -1.02, 106.7, -39.18, -13.37]
servo in:  106 92 87 105 83 83 3
[-10.83, -2.2, 8.16, 22.8, 51.3, 34.54]
servo in:  82 97 96 116 96 100 3
[-9.26, 6.08, -6.12, -63.25, -104.59, -19.84]
servo in:  99 62 81 97 82 90 3
[-7.77, -0.59, 3.92, 22.66, 44.99, -47.76]
servo in:  87 100 96 101 85 88 3
[-12.12, 1.06, 14.04, -58.57, -29.27, 6.62]
servo in:  92 80 87 87 99 92 3
[-9.1, -3.26, 1.26, 29.05, -17.68, -76.1]
servo in:  117 89 98 95 59 86 3
[-3.73, -6.43, 0.47, 80.49, 246.06, 69.71]
I have fallen.

[-10.83, -5.18, 1.57, -54.4, -190.09, -87.35]
servo in:  90 90 90 90 90 90 3
Steps to fall :7
Steps to fall:7

[Upd]sensor.txt
Sensor data recieved.



In [6]:
ser.close()